# Test quality of AI Search Context for RAG

In [2]:
import pandas as pd
import numpy as numpy
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import openai, os, requests
import time
import pprint

pp=pprint.PrettyPrinter()

%load_ext dotenv
%dotenv

In [3]:
print('OpenAI version = ', openai.__version__)

OpenAI version =  0.28.0


In [4]:


openai.api_type = "azure"
# Azure OpenAI on your own data is only supported by the 2023-08-01-preview API version
openai.api_version = "2023-08-01-preview"

# Azure OpenAI setup
openai.api_base = "https://aoai-cogsearch-test-ep.openai.azure.com/" # Add your endpoint here
openai.api_key = os.getenv("OPENAI_API_KEY") # Add your OpenAI API key here
deployment_id = "gpt-4" # Add your deployment ID here

# Azure AI Search setup
search_endpoint = "https://ai-cog-search-telstra.search.windows.net"; # Add your Azure AI Search endpoint here
search_key = os.getenv("AI_SEARCH_KEY"); # Add your Azure AI Search admin key here
search_index_name = "telstra-vector-index"; # Add your Azure AI Search index name here

SYSTEM_PROMPT = """
You are an AI bot designed to answer questions about Telstra. 
You must respond only from the data source provided. 
If you do not know the answer, you can say 'I don't know'.
"""

In [5]:

def setup_byod(deployment_id: str) -> None:
    """Sets up the OpenAI Python SDK to use your own data for the chat endpoint.

    :param deployment_id: The deployment ID for the model to use with your own data.

    To remove this configuration, simply set openai.requestssession to None.
    """

    class BringYourOwnDataAdapter(requests.adapters.HTTPAdapter):

        def send(self, request, **kwargs):
            request.url = f"{openai.api_base}/openai/deployments/{deployment_id}/extensions/chat/completions?api-version={openai.api_version}"
            return super().send(request, **kwargs)

    session = requests.Session()

    # Mount a custom adapter which will use the extensions endpoint for any call using the given `deployment_id`
    session.mount(
        prefix=f"{openai.api_base}/openai/deployments/{deployment_id}",
        adapter=BringYourOwnDataAdapter()
    )

    openai.requestssession = session

setup_byod(deployment_id)

In [6]:
BASE_PROMPT = [
    {"role":"system", "content": SYSTEM_PROMPT},
    ]

In [7]:
def print_chat_history(chat_history):
    for message in chat_history:
        pp.pprint(f"{message['role']}: {message['content']}")

def get_chat_response(chat_history,
                      temperature=0.7,
                        max_tokens=150,
                        top_p=1,
                        frequency_penalty=0,
                        presence_penalty=0,
                        seed=12345,
                        topN=5,
                        strictness=3,
                        enforce_inscope=True,
                        queryType="vector", # simple, semantic, vectorSimpleHybrid, vectorSemanticHybrid
                        roleInformation=SYSTEM_PROMPT,
                      ):
    completion = openai.ChatCompletion.create(
        temperature=0.7,
        messages=chat_history,
        deployment_id=deployment_id,
        dataSources=[  # camelCase is intentional, as this is the format the API expects
            {
                "type": "AzureCognitiveSearch",
                "parameters": {
                    "endpoint": search_endpoint,
                    "key": search_key,
                    "indexName": search_index_name,
                    "topNDocuments": topN,
                    "inScope": enforce_inscope,
                    "semanticConfiguration": "semantic-config-a0859498-994",
                    "roleInformation": roleInformation,
                    "strictness": strictness,
                    "fieldsMapping": {
                        "contentFields": [
                            "chunk",
                        ],
                        # "titleField" : "ADD TITLE TO INDEX",
                        # "urlField" : "ADD URL TO INDEX",
                        # "filepathField" : "ADD FILEPATH TO INDEX",
                    }

                }
            }
        ]
    )
    return completion

In [8]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from copy import deepcopy
def get_user_input():
    # initialize chat history
    chat_history = deepcopy(BASE_PROMPT)
    context_history = []
    # Create text input widget
    text_input = widgets.Textarea(
        value='',
        placeholder='Type something',
        description='User Input:',
        disable=False
    )

    # Create a button widget for submitting
    submit_button = widgets.Button(
        description='Submit',
        disable=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Submit',
        icon='check' # (FontAwesome names without the `fa-` prefix)
    )

    # Create a button widget for clearing
    clear_button = widgets.Button(
        description='Clear',
        disable=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Clear',
        icon='remove' # (FontAwesome names without the `fa-` prefix)
    )

    # Display the widgets
    display(text_input, submit_button, clear_button)

    # Function to handle the submit button click event
    def on_submit_button_clicked(b):
        # Get the text input
        user_input = text_input.value

        # construct the prompt
        chat_history.append({"role":"user", "content": user_input})

        # Pass the input to the get_chat_response function
        start=time.time()
        response = get_chat_response(chat_history)
        # response = "DUMMY RESPONSE"
        end=time.time()
        # bot response
        bot_response = response.choices[0]['message']['content']
        context_history.append(response.choices[0]['message']['context']['messages'][0]['content'])

        # append the bot response to the chat history
        chat_history.append({"role":"assistant", "content": bot_response})

        # print chat history
        print_chat_history(chat_history)
        pp.pprint(context_history)
        pp.pprint(f"Time taken: {end-start:.3} seconds")

    # Function to handle the clear button click event
    def on_clear_button_clicked(b):
        # Clear the output
        clear_output(wait=False)
        display(text_input, submit_button, clear_button)


    # Attach the event handlers to the button widgets
    submit_button.on_click(on_submit_button_clicked)
    clear_button.on_click(on_clear_button_clicked)

In [9]:
get_user_input()

Textarea(value='', description='User Input:', placeholder='Type something')

Button(description='Submit', icon='check', style=ButtonStyle(), tooltip='Submit')

Button(description='Clear', icon='remove', style=ButtonStyle(), tooltip='Clear')

In [10]:
QUESTION_BANK = [
    "How do I check the expiry of my SIM card",
    "From which year are the Telstra financial reports available?",
    "How do I buy or sell Telstra shares?",
    "Can sharehholders receive a discount o Telstra products. Answer in yes or no",
    "What is Telstra's sustainability strategy?",
    "How does Telstra work on Digital Literacy?",
    "What does LIMAC stand for and how is it related to access for everyone?",
    "Which edition of the ASX Corporate Governance Principles and Recommendations does Telstra follow?",
    "When did Telstra Group become the new listed entity of the Telstra Corporation Limited?",
    "How do I pay my bills on the My Telstra app? Answer in bullet point format in 1-2 sentences",
]

In [70]:
results = []
for qType in ["vector","simple", "semantic","vectorSimpleHybrid", "vectorSemanticHybrid"]:
# for qType in ["vectorSemanticHybrid"]:
    for q in QUESTION_BANK:
        start = time.time()
        chat_history = deepcopy(BASE_PROMPT)
        chat_history.append({"role":"user", "content": q})
        response = get_chat_response(chat_history, queryType=qType)
        end=time.time()
        bot_response = response.choices[0]['message']['content']
        context_history = response.choices[0]['message']['context']['messages'][0]['content']
        chat_history.append({"role":"assistant", "content": bot_response})
        print_chat_history(chat_history)
        pp.pprint(f"Time taken: {end-start:.3} seconds")
        results.append([qType, q, bot_response, context_history, end-start])

('system: \n'
 'You are an AI bot designed to answer questions about Telstra. \n'
 'You must respond only from the data source provided. \n'
 "If you do not know the answer, you can say 'I don't know'.\n")
'user: How do I check the expiry of my SIM card'
('assistant: To check the expiry of your Telstra SIM card, you can visit the '
 "Telstra website at m.telstra.com from your device's browser or login to "
 'MyAccount at telstra.com/myaccount. Additionally, you can use the Telstra '
 '24x7® App on iPhone and Android to monitor and manage your usage, including '
 'checking the expiry of your SIM card[doc5]. If you have a Pre-Paid SIM card, '
 'remember that any unused data will expire at the end of your recharge period '
 'unless you have eligible Rollover Data, which expires if you change offers '
 'or if your service is deactivated[doc5].')
'Time taken: 27.6 seconds'
('system: \n'
 'You are an AI bot designed to answer questions about Telstra. \n'
 'You must respond only from the data

In [66]:
# df = pd.DataFrame(results, columns = ["query_type", "question","response","context", "time_taken"])
# df["time_taken"].describe()


count    44.000000
mean     22.263432
std      14.627290
min       5.106458
25%      10.649173
50%      15.824201
75%      32.172456
max      51.856385
Name: time_taken, dtype: float64

In [71]:
df = df[df['query_type'] != 'vectorSemanticHybrid']
df2 = pd.DataFrame(results, columns = ["query_type", "question","response","context", "time_taken"])
df = pd.concat([df, df2])

In [72]:
df.groupby("query_type").agg({"time_taken": ["mean", "std"]}).sort_values(("time_taken", "mean"))

time_taken           
                           mean        std
query_type                                
semantic              19.716659  11.414389
vectorSimpleHybrid    21.255067  14.021796
vector                23.606304  16.426277
vectorSemanticHybrid  24.195999  16.342318
simple                25.201094  17.830180

In [73]:
df.to_csv("results.csv", index=False)